In [ ]:
# Install necessary packages
!pip install tqdm torch torchvision pytorch-msssim

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from tqdm import tqdm
from pathlib import Path
from PIL import Image
import random
import numpy as np
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
from pytorch_msssim import ssim
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.dirname(os.getcwd()))

from models.autoencoder.auto import Auto  # Import your custom Auto class

# ✅ Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Paths
DATA_DIR = Path("../data/SD1")
PROCESSED_TRAIN_DIR = DATA_DIR / "processed_train"
PROCESSED_VAL_DIR = DATA_DIR / "processed_val"
MODEL_DIR = Path("../models")
MODEL_DIR.mkdir(parents=True, exist_ok=True)  # Ensure model directory exists

# ✅ Dataset class
class GlareRemovalDataset(Dataset):
    def __init__(self, root_dir: str, transform: transforms.Compose = None, limit: int = None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.image_pairs = []

        glare_images = sorted(self.root_dir.glob("glare_*.png"))
        gt_images = sorted(self.root_dir.glob("gt_*.png"))

        for glare_img in glare_images:
            gt_img = self.root_dir / glare_img.name.replace("glare_", "gt_")
            if gt_img.exists():
                self.image_pairs.append((glare_img, gt_img))

        if limit:
            self.image_pairs = random.sample(self.image_pairs, min(limit, len(self.image_pairs)))

    def __len__(self) -> int:
        return len(self.image_pairs)

    def __getitem__(self, idx: int):
        glare_path, gt_path = self.image_pairs[idx]
        glare_img = Image.open(glare_path).convert("RGB")
        gt_img = Image.open(gt_path).convert("RGB")

        if self.transform:
            glare_img = self.transform(glare_img)
            gt_img = self.transform(gt_img)

        return glare_img, gt_img

# ✅ Transformations with data augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
])

# ✅ Data loaders (Optimized)
train_dataset = GlareRemovalDataset(PROCESSED_TRAIN_DIR, transform=transform)
val_dataset = GlareRemovalDataset(PROCESSED_VAL_DIR, transform=transforms.ToTensor())

train_loader = DataLoader(
    train_dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=16, shuffle=False, num_workers=0, pin_memory=True
)

# ✅ Instantiate Model
model = Auto().to(device)

# ✅ Custom Loss (L1 + SSIM Loss)
def custom_loss(predicted: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    l1_loss = F.l1_loss(predicted, target)
    ssim_loss = 1 - ssim(predicted, target, data_range=1, size_average=True)
    return 0.5 * l1_loss + 0.5 * ssim_loss

# ✅ Optimizer & Scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)

# ✅ Training loop
num_epochs = 30  # Increased for better learning
best_val_loss = float("inf")
patience = 15
counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, targets = images.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = custom_loss(outputs, targets)
        loss.backward()

        # ✅ Gradient Clipping to avoid exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Training Loss: {avg_train_loss:.4f}")

    # ✅ Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            val_loss += custom_loss(outputs, targets).item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1} - Validation Loss: {avg_val_loss:.4f}")

    # ✅ Adjust learning rate
    scheduler.step()

    # ✅ Early stopping and model saving
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        best_model_path = MODEL_DIR / "best_glare_removal_autoencoder_augmented.pth"
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Best Model Saved at: {best_model_path}")
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break

    # ✅ Periodic checkpoint saving
    if (epoch + 1) % 10 == 0:
        checkpoint_path = MODEL_DIR / f"glare_autoencoder_augmented_epoch_{epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_path)
        print(f"📌 Checkpoint saved at: {checkpoint_path}")

# ✅ Save final model
final_model_path = MODEL_DIR / "final_glare_removal_autoencoder_augemented.pth"
torch.save(model.state_dict(), final_model_path)
print(f"🎉 Final Model Saved at: {final_model_path}")


Using device: cpu


Epoch 1/30: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [15:02<00:00,  1.20s/it]


Epoch 1 - Training Loss: 0.3755
Epoch 1 - Validation Loss: 0.2316
✅ Best Model Saved at: ..\models\best_glare_removal_autoencoder_augmented.pth


Epoch 2/30: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [15:09<00:00,  1.21s/it]


Epoch 2 - Training Loss: 0.2482
Epoch 2 - Validation Loss: 0.2239
✅ Best Model Saved at: ..\models\best_glare_removal_autoencoder_augmented.pth


...

Epoch 22/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [25:45<00:00,  2.06s/it]


Epoch 22 - Training Loss: 0.2388
Epoch 22 - Validation Loss: 0.2133


Epoch 23/30: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 750/750 [29:43<00:00,  2.38s/it]


Epoch 23 - Training Loss: 0.2386
Epoch 23 - Validation Loss: 0.2122


Epoch 24/30:   1%|█▌                                                                                                                                                                          | 7/750 [00:16<29:28,  2.38s/it]